In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
!pip install feature_engine
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import warnings, gc, joblib, re
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression, PassiveAggressiveClassifier, RidgeClassifier, SGDClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier, BaggingClassifier, VotingClassifier, StackingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, PrecisionRecallDisplay, RocCurveDisplay
from scipy.stats import probplot
from feature_engine.outliers import Winsorizer
from feature_engine.selection import DropConstantFeatures, DropCorrelatedFeatures, DropDuplicateFeatures
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, SelectPercentile, SelectFromModel, chi2, RFE, SequentialFeatureSelector
from xgboost import XGBClassifier, XGBRFClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from imblearn.over_sampling import BorderlineSMOTE
from collections import Counter
from yellowbrick.classifier import ClassPredictionError

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

  Obtaining dependency information for feature_engine from https://files.pythonhosted.org/packages/62/60/77fcc9d3cfaabab34027aa8ea0025c5e2d4cf9561fa9725a38f0785b43aa/feature_engine-1.6.2-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.9/328.9 kB 10.1 MB/s eta 0:00:00
/kaggle/input/company-bankruptcy-prediction/data.csv


In [2]:
df=pd.read_csv('/kaggle/input/company-bankruptcy-prediction/data.csv')
data=df.copy()
df.head()

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
0,1,0.370594,0.424389,0.405750,0.601457,0.601457,0.998969,0.796887,0.808809,0.302646,...,0.716845,0.009219,0.622879,0.601453,0.827890,0.290202,0.026601,0.564050,1,0.016469
1,1,0.464291,0.538214,0.516730,0.610235,0.610235,0.998946,0.797380,0.809301,0.303556,...,0.795297,0.008323,0.623652,0.610237,0.839969,0.283846,0.264577,0.570175,1,0.020794
2,1,0.426071,0.499019,0.472295,0.601450,0.601364,0.998857,0.796403,0.808388,0.302035,...,0.774670,0.040003,0.623841,0.601449,0.836774,0.290189,0.026555,0.563706,1,0.016474
3,1,0.399844,0.451265,0.457733,0.583541,0.583541,0.998700,0.796967,0.808966,0.303350,...,0.739555,0.003252,0.622929,0.583538,0.834697,0.281721,0.026697,0.564663,1,0.023982
4,1,0.465022,0.538432,0.522298,0.598783,0.598783,0.998973,0.797366,0.809304,0.303475,...,0.795016,0.003878,0.623521,0.598782,0.839973,0.278514,0.024752,0.575617,1,0.035490


In [3]:
print('Bankrupt: {},No Bankrupt:{}'.format((df['Bankrupt?']==1).sum(),(df['Bankrupt?']==0).sum()))

Bankrupt: 220,No Bankrupt:6599


We need to perform oversampling or undersampling as the classes are really imbalanced. So we use SMOTE technique

In [4]:
def check_basics(dataframe):
    print(dataframe.shape)
    print(dataframe.dtypes)
    print(dataframe.isnull().sum())
check_basics(df)    

(6819, 96)
Bankrupt?                                                     int64
 ROA(C) before interest and depreciation before interest    float64
 ROA(A) before interest and % after tax                     float64
 ROA(B) before interest and depreciation after tax          float64
 Operating Gross Margin                                     float64
                                                             ...   
 Liability to Equity                                        float64
 Degree of Financial Leverage (DFL)                         float64
 Interest Coverage Ratio (Interest expense to EBIT)         float64
 Net Income Flag                                              int64
 Equity to Liability                                        float64
Length: 96, dtype: object
Bankrupt?                                                   0
 ROA(C) before interest and depreciation before interest    0
 ROA(A) before interest and % after tax                     0
 ROA(B) before interest and d

In [5]:
df.drop(df.columns[df.nunique()==1][0],axis=1,inplace=True)
df.columns=df.columns.str.replace(' ','')

Next we find out categorical columns and numerical columns as well as columns with low cardinality. This is done to ensure that if some numerical values are represented in categotical values but with numbers, they can be classified. We also remove columns with low unique values. 

In [6]:
def clean_columns(dataframe,cat_th=10, car_th=20):
    cat_cols=[col for col in dataframe.columns if dataframe[col].dtypes=="0" ]
    num_but_cat=[col for col in dataframe.columns if dataframe[col].dtypes!="0" and dataframe[col].nunique()<cat_th]
    cat_but_car=[col for col in dataframe.columns if dataframe[col].dtypes=="0" and dataframe[col].nunique()>car_th]
    cat_cols=cat_cols+num_but_cat
    cat_cols=[col for col in cat_cols if col not in cat_but_car]
    num_cols=[col for col in dataframe.columns if dataframe[col].dtypes!="0"]
    num_cols=[col for col in num_cols if col not in num_but_cat]
    print(f"Observations: {dataframe.shape[0]}")
    print(f"Variables: {dataframe.shape[1]}")
    print(f'cat_cols: {len(cat_cols)}')
    print(f'num_cols: {len(num_cols)}')
    print(f'cat_but_car: {len(cat_but_car)}')
    print(f'num_but_cat: {len(num_but_cat)}')
    return cat_cols, cat_but_car, num_cols
    
cat_cols,cat_but_car,num_cols=clean_columns(df,10,20)    

Observations: 6819
Variables: 95
cat_cols: 2
num_cols: 93
cat_but_car: 0
num_but_cat: 2


Next we process the outliers. For finding and setting the values of threshold we use IQR as it is more robust against outliers. We replace outliers with threshold values.

In [7]:
def find_threshold(dataframe,variable,low_quantile=0.15,up_quantile=0.85):
    quantile_one=dataframe[variable].quantile(low_quantile)
    quantile_three=dataframe[variable].quantile(up_quantile)
    iqr=quantile_three-quantile_one
    low_limit=quantile_one-(1.5*iqr)
    up_limit=quantile_three+(1.5*iqr)
    return low_limit, up_limit

def check_outliers(dataframe, col_name):
        low_limit,up_limit=find_threshold(df,col_name)
        if dataframe[(dataframe[col_name]>up_limit) | (dataframe[col_name]<low_limit)].any(axis=None):
            return True
        else:
            return False
for col in num_cols:
    print(check_outliers(df,col))

True
True
True
True
True
True
True
True
True
True
False
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
True
True
True
True
True
False
False
True
True
True
True
True
True
True
False
True
True
True
True
True
True
True
False
True
False
True
True
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [8]:
def replace_with_threshold(dataframe, col_name):
    low_limit, up_limit=find_threshold(dataframe,col_name)
    dataframe.loc[dataframe[col_name]>up_limit,col_name]=up_limit
    dataframe.loc[dataframe[col_name]<low_limit,col_name]=low_limit
for col in num_cols:
        print(col)
        if(col!="SalePrice"):
            replace_with_threshold(df,col)
    
    

ROA(C)beforeinterestanddepreciationbeforeinterest
ROA(A)beforeinterestand%aftertax
ROA(B)beforeinterestanddepreciationaftertax
OperatingGrossMargin
RealizedSalesGrossMargin
OperatingProfitRate
Pre-taxnetInterestRate
After-taxnetInterestRate
Non-industryincomeandexpenditure/revenue
Continuousinterestrate(aftertax)
OperatingExpenseRate
Researchanddevelopmentexpenserate
Cashflowrate
Interest-bearingdebtinterestrate
Taxrate(A)
NetValuePerShare(B)
NetValuePerShare(A)
NetValuePerShare(C)
PersistentEPSintheLastFourSeasons
CashFlowPerShare
RevenuePerShare(Yuan¥)
OperatingProfitPerShare(Yuan¥)
PerShareNetprofitbeforetax(Yuan¥)
RealizedSalesGrossProfitGrowthRate
OperatingProfitGrowthRate
After-taxNetProfitGrowthRate
RegularNetProfitGrowthRate
ContinuousNetProfitGrowthRate
TotalAssetGrowthRate
NetValueGrowthRate
TotalAssetReturnGrowthRateRatio
CashReinvestment%
CurrentRatio
QuickRatio
InterestExpenseRatio
Totaldebt/Totalnetworth
Debtratio%
Networth/Assets
Long-termfundsuitabilityratio(A)
Borrowin

In [9]:
y=df['Bankrupt?']
X=df.drop('Bankrupt?',axis=1)


In [10]:
pipeline=Pipeline(steps=[('constant',DropConstantFeatures()),
                        ('correlated',DropCorrelatedFeatures()),
                        ('duplicate',DropDuplicateFeatures())])
X=pipeline.fit_transform(X)
X.shape

(6819, 54)

In [11]:
from sklearn.preprocessing import RobustScaler

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=42, stratify=y)
scale=StandardScaler()
scaled_train_X=scale.fit_transform(X_train)

In [12]:
from sklearn.decomposition import PCA

pca=PCA()
pca.fit(scaled_train_X)
exp_variance=pca.explained_variance_ratio_
cumulative_exp_variance=np.cumsum(exp_variance)
print(cumulative_exp_variance)

pca=PCA(n_components=54,random_state=7)
train_pca=pca.fit_transform(scaled_train_X)

scaled_test_X=scale.fit_transform(X_test)
test_pca=pca.fit_transform(scaled_test_X)

[0.1596778  0.2606052  0.34743063 0.39863184 0.44687238 0.48538984
 0.52089729 0.5545121  0.58576535 0.61121715 0.6339548  0.65504989
 0.67443827 0.69349594 0.71176888 0.7290693  0.74592486 0.76157183
 0.77650715 0.79131313 0.80556592 0.81907907 0.83239693 0.84458842
 0.85659243 0.86794833 0.87882667 0.88885291 0.89859906 0.90694428
 0.91469062 0.92185196 0.9289009  0.93561165 0.94206673 0.94760331
 0.95275339 0.95762626 0.96240875 0.96660042 0.97071104 0.97462841
 0.97817073 0.98139321 0.984443   0.98735434 0.99008344 0.99226952
 0.99435924 0.99610517 0.99784282 0.99905458 0.99974338 1.        ]


In [13]:
models=[]
accuracy=[]
f1=[]
precision=[]
recall=[]
roc_auc=[]

def train_and_evaluate_model(model):
    model.fit(train_pca, y_train)
    y_pred=model.predict(test_pca)
    print("Classification Report:")
    print(classification_report(y_test,y_pred))
    print("-"*50)
    

